# Group by and function in SQLAlchemy
***

From the previous we know how create query to database in SQLAlchemy based on function [select](https://docs.sqlalchemy.org/en/13/core/metadata.html?highlight=select#sqlalchemy.schema.Table.select) or [query](https://docs.sqlalchemy.org/en/14/orm/query.html)

The purpose of these laboratory classes is to familiarize participants with group by and function in SQLAlchemy.

The scope of this classes:
 - using group_by()
 - using func()

The GROUP BY clause divides the rows returned from the SELECT statement into groups. For each group, you can apply an aggregate function e.g.,  SUM() to calculate the sum of items or COUNT() to get the number of items in the groups.

Of course, if we need filtering result of aggregation we can use function [having](https://docs.sqlalchemy.org/en/14/orm/query.html#sqlalchemy.orm.Query.having):

SQLAlchemy's [func](https://docs.sqlalchemy.org/en/14/core/functions.html) module provides access to built-in SQL functions that can make operations like counting and summing faster and more efficient.

### Exercise

Use all of these methods to create queries for the test database. Check their execution time using the [profiling and timing code methods](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html).

For queries:
1. Calculate the average cost of renting out all your movies.
2. Calculate and display the number of videos in all categories.
3. View the count of all customers grouped by country.
4. Display information about a store that has more than 100 customers and less than 300 customers.
5. Select all customers who watched movies for more than 200 hours.
6. Calculate the average value of a movie rented.
7. Calculate the average value of the video length in all categories.
8. Find longest movie titles in all categories.
9. Find longest movie in all categories. Compare the result with point 10.

![schema dvd rental](dvd-rental-sample-database-diagram.png)

## Prepare the environment

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

In [2]:
config_PostgreSQL = {
    "database_type": "",
    "user": "",
    "password": "",
    "database_url": "",
    "port": ,
    "database_name": ""
}

db_string = "{database_type}://{user}:{password}@{database_url}:{port}/{database_name}".format(**config_PostgreSQL)

engine = create_engine(db_string)

# test the connection
try:
    conn = engine.connect()
    print("Connected successfully!")
except Exception as e:
    print("Failed to connect")
    print(f"Error: {e}")

Base = declarative_base()

Connected successfully!


### Initialize mapper operation

We can use a script to initialize mapper operation. Where `dict_table` is the dictionary with tables representation where the key is the name of the table.

In [3]:
from sqlalchemy import MetaData, Table, select, and_
from sqlalchemy.inspection import inspect

metadata = MetaData()
dict_table = {}

inspector = inspect(engine)

for table_name in inspector.get_table_names():
    dict_table[table_name] = Table(table_name, metadata, autoload_with=engine)

The first part of the laboratory will concern the case of working with a database whose structure is don't well known.

In [4]:
from sqlalchemy.orm import sessionmaker

session = (sessionmaker(bind=engine))()

Base = declarative_base()

### Define database object representation

The first part of the laboratory will concern the case of working with a database whose structure is don't well known.

All the examples for this laboratory part will be for the tables that are mapped on the classes (Object representation):

In [5]:
from sqlalchemy import Column, Integer, String, Date, ForeignKey, PrimaryKeyConstraint, Float, SmallInteger, select, func
from sqlalchemy.orm import relationship, declarative_base

class Country(Base):
    __tablename__ = 'country'
    country_id = Column(Integer, primary_key=True)
    country = Column(String(50))
    last_update = Column(Date)
    # Relationships
    cities = relationship("City", back_populates="country")
    
    def __str__(self):
        return f"Country id: {self.country_id}, Country name: {self.country}, Last update: {self.last_update}"

class City(Base):
    __tablename__ = 'city'
    city_id = Column(Integer, primary_key=True)
    city = Column(String(50))
    country_id = Column(Integer, ForeignKey('country.country_id'))
    last_update = Column(Date)
    # Relationships
    country = relationship("Country", back_populates="cities")
    addresses = relationship("Address", back_populates="city")
    
    def __str__(self):
        return f"City id: {self.city_id}, City name: {self.city}, Country id: {self.country_id}, Last update: {self.last_update}"

class Store(Base):
    __tablename__ = 'store'
    store_id = Column(Integer, primary_key=True)
    manager_staff_id = Column(Integer, ForeignKey('staff.staff_id'))
    address_id = Column(Integer, ForeignKey('address.address_id'))
    last_update = Column(Date)

    address = relationship("Address", back_populates="stores")
    staff_members = relationship("Staff", back_populates="store", foreign_keys=[manager_staff_id])
    customers = relationship("Customer", back_populates="store")

    def __str__(self):
        return f"Store id: {self.store_id}, Manager staff id: {self.manager_staff_id}, Address id: {self.address_id}, Last update: {self.last_update}"

class Staff(Base):
    __tablename__ = 'staff'
    staff_id = Column(Integer, primary_key=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    address_id = Column(Integer, ForeignKey('address.address_id'))
    email = Column(String(50))
    store_id = Column(Integer, ForeignKey('store.store_id'))
    active = Column(Integer)
    username = Column(String(50))
    password = Column(String(50))
    last_update = Column(Date)
    picture = Column(String)

    store = relationship("Store", back_populates="staff_members", foreign_keys=[store_id])

    def __str__(self):
        return f"Staff id: {self.staff_id}, Name: {self.first_name} {self.last_name}, Store id: {self.store_id}, Last update: {self.last_update}"

class Address(Base):
    __tablename__ = 'address'

    address_id = Column(Integer, primary_key=True)
    address = Column(String(50))
    address2 = Column(String(50))
    district = Column(String(20))
    city_id = Column(Integer, ForeignKey('city.city_id'))
    postal_code = Column(String(10))
    phone = Column(String(20))
    last_update = Column(Date)
    # Relationships
    city = relationship("City", back_populates="addresses")
    customers = relationship("Customer", back_populates="address")
    stores = relationship("Store", back_populates="address")

    def __str__(self):
        return f"Address id: {self.address_id}, Address: {self.address}, City id: {self.city_id}, Last update: {self.last_update}"

class Customer(Base):
    __tablename__ = 'customer'
    customer_id = Column(Integer, primary_key=True)
    store_id = Column(Integer, ForeignKey('store.store_id'))
    first_name = Column(String(45))
    last_name = Column(String(45))
    email = Column(String(50))
    address_id = Column(Integer, ForeignKey('address.address_id'))
    activebool = Column(Integer)
    create_date = Column(Date)
    last_update = Column(Date)
    active = Column(Integer)
    # Relationships
    address = relationship("Address", back_populates="customers")
    store = relationship("Store", back_populates="customers")
    rentals = relationship("Rental", back_populates="customer")

    def __str__(self):
        return (f"Customer id: {self.customer_id}, Name: {self.first_name} {self.last_name}, "
                f"Email: {self.email}, Store id: {self.store_id}, Last update: {self.last_update}")

class Rental(Base):
    __tablename__ = 'rental'
    rental_id = Column(Integer, primary_key=True)
    rental_date = Column(Date)
    inventory_id = Column(Integer, ForeignKey('inventory.inventory_id'))
    customer_id = Column(Integer, ForeignKey('customer.customer_id'))
    return_date = Column(Date)
    staff_id = Column(Integer, ForeignKey('staff.staff_id'))
    last_update = Column(Date)
    # Relationships
    inventory = relationship("Inventory", back_populates="rentals")
    customer = relationship("Customer", back_populates="rentals")
    staff = relationship("Staff", back_populates="rentals")

    def __str__(self):
        return (f"Rental id: {self.rental_id}, Rental date: {self.rental_date}, "
                f"Inventory id: {self.inventory_id}, Customer id: {self.customer_id}, Last update: {self.last_update}")

class Inventory(Base):
    __tablename__ = 'inventory'
    inventory_id = Column(Integer, primary_key=True)
    film_id = Column(Integer, ForeignKey('film.film_id'))
    store_id = Column(Integer, ForeignKey('store.store_id'))
    last_update = Column(Date)
    
    film = relationship("Film", back_populates="inventories")
    rentals = relationship("Rental", back_populates="inventory")

    def __str__(self):
        return f"Inventory id: {self.inventory_id}, Film id: {self.film_id}, Last update: {self.last_update}"

class Language(Base):
    __tablename__ = 'language'
    language_id = Column(Integer, primary_key=True)
    name = Column(String(20))
    last_update = Column(Date)
    
    films = relationship("Film", back_populates="language")

    def __str__(self):
        return (f"Language id: {self.language_id}, Name: {self.name}, Last update: {self.last_update}")

class Category(Base):
    __tablename__ = 'category'

    category_id = Column(Integer, primary_key=True)
    name = Column(String(255))
    last_update = Column(Date)
    
    films = relationship("FilmCategory", back_populates="category")

    def __str__(self):
        return f"Category id: {self.category_id}, Name: {self.name}, Last update: {self.last_update}"

class FilmCategory(Base):
    __tablename__ = 'film_category'

    film_id = Column(Integer, ForeignKey('film.film_id'), primary_key=True)
    category_id = Column(Integer, ForeignKey('category.category_id'), primary_key=True)
    last_update = Column(Date)

    film = relationship("Film", back_populates="film_categories")
    category = relationship("Category", back_populates="films")

    def __str__(self):
        return (f"Film ID: {self.film_id}, Category ID: {self.category_id}, Last update: {self.last_update}")

class Film(Base):
    __tablename__ = 'film'
    film_id = Column(Integer, primary_key=True)
    title = Column(String(255))
    description = Column(String(255))
    release_year = Column(Integer)
    language_id = Column(Integer, ForeignKey('language.language_id'))
    rental_duration = Column(Integer)
    rental_rate = Column(Float)
    length = Column(Integer)
    replacement_cost = Column(Float)
    rating = Column(String(10))
    last_update = Column(Date)
    special_features = Column(String(255))
    fulltext = Column(String(255))

    language = relationship("Language", back_populates="films")
    film_categories = relationship("FilmCategory", back_populates="film")
    inventories = relationship("Inventory", back_populates="film")

    def __str__(self):
        return (f"Film id: {self.film_id}, Title: {self.title}, "
                f"Rental rate: {self.rental_rate}, Last update: {self.last_update}")

class Actor(Base):
    __tablename__ = 'actor'
    actor_id = Column(Integer, primary_key=True)
    first_name = Column(String(45))
    last_name = Column(String(45))
    last_update  = Column(Date) 

class Film_actor(Base):
    __tablename__ = 'film_actor'
    actor_id = Column(Integer,  ForeignKey('actor.actor_id'))
    film_id = Column(Integer,  ForeignKey('film.film_id'))
    PrimaryKeyConstraint(actor_id,film_id)
    last_update  = Column(Date)

## 1. Calculate the average cost of renting out all your movies.

Core API:

In [6]:
mapper_stmt1 = select(func.avg(dict_table['film'].c.rental_rate).label('average_rental_rate'))

print('Mapper select: ')
print(mapper_stmt1)

with engine.connect() as conn:
    result1 = conn.execute(mapper_stmt1).fetchall()

print(result1)

Mapper select: 
SELECT avg(film.rental_rate) AS average_rental_rate 
FROM film
[(Decimal('2.9800000000000000'),)]


ORM API:

In [7]:
session_stmt1 = session.query(func.avg(dict_table['film'].c.rental_rate).label('average_rental_rate'))

print('\nSession select: ')
print(session_stmt1)

result2 = session_stmt1.all()

print(result1)


Session select: 
SELECT avg(film.rental_rate) AS average_rental_rate 
FROM film
[(Decimal('2.9800000000000000'),)]


## 2. Calculate and display the number of videos in all categories.

In [8]:
mapper_stmt2 = select(
    dict_table['category'].c.name.label('category_name'),
    func.count(dict_table['film_category'].c.film_id).label('film_count')
).select_from(
    dict_table['film_category'].join(dict_table['category'], dict_table['film_category'].c.category_id == dict_table['category'].c.category_id)
).group_by(
    dict_table['category'].c.name
)

print('Mapper select: ')
print(mapper_stmt2)

with engine.connect() as conn:
    result2 = conn.execute(mapper_stmt2).fetchall()

print(result2)

Mapper select: 
SELECT category.name AS category_name, count(film_category.film_id) AS film_count 
FROM film_category JOIN category ON film_category.category_id = category.category_id GROUP BY category.name
[('Family', 69), ('Games', 61), ('Animation', 66), ('Classics', 57), ('Documentary', 68), ('New', 63), ('Sports', 74), ('Children', 60), ('Music', 51), ('Travel', 57), ('Foreign', 73), ('Drama', 62), ('Horror', 56), ('Action', 64), ('Sci-Fi', 61), ('Comedy', 58)]


In [9]:
session_stmt2 = session.query(
    dict_table['category'].c.name.label('category_name'),
    func.count(dict_table['film_category'].c.film_id).label('film_count')
).join(
    dict_table['film_category'], dict_table['film_category'].c.category_id == dict_table['category'].c.category_id
).group_by(
    dict_table['category'].c.name
)

print('\nSession select: ')
print(session_stmt2)

result2 = session_stmt2.all()

print(result2)


Session select: 
SELECT category.name AS category_name, count(film_category.film_id) AS film_count 
FROM category JOIN film_category ON film_category.category_id = category.category_id GROUP BY category.name
[('Family', 69), ('Games', 61), ('Animation', 66), ('Classics', 57), ('Documentary', 68), ('New', 63), ('Sports', 74), ('Children', 60), ('Music', 51), ('Travel', 57), ('Foreign', 73), ('Drama', 62), ('Horror', 56), ('Action', 64), ('Sci-Fi', 61), ('Comedy', 58)]


## 3. View the count of all customers grouped by country.

In [10]:
mapper_stmt3 = select(
    dict_table['country'].c.country.label('country_name'),
    func.count(dict_table['customer'].c.customer_id).label('customer_count')
).select_from(
    dict_table['customer']
    .join(dict_table['address'], dict_table['customer'].c.address_id == dict_table['address'].c.address_id)
    .join(dict_table['city'], dict_table['address'].c.city_id == dict_table['city'].c.city_id)
    .join(dict_table['country'], dict_table['city'].c.country_id == dict_table['country'].c.country_id)
).group_by(
    dict_table['country'].c.country
)

print('Mapper select: ')
print(mapper_stmt3)

with engine.connect() as conn:
    result3 = conn.execute(mapper_stmt3).fetchall()

print(result3)

Mapper select: 
SELECT country.country AS country_name, count(customer.customer_id) AS customer_count 
FROM customer JOIN address ON customer.address_id = address.address_id JOIN city ON address.city_id = city.city_id JOIN country ON city.country_id = country.country_id GROUP BY country.country
[('Bangladesh', 3), ('Indonesia', 14), ('Venezuela', 7), ('Cameroon', 2), ('Czech Republic', 1), ('Sweden', 1), ('Dominican Republic', 3), ('Cambodia', 2), ('Sri Lanka', 1), ('American Samoa', 1), ('Brunei', 1), ('Finland', 1), ('Colombia', 6), ('Ukraine', 6), ('Saudi Arabia', 5), ('Latvia', 2), ('North Korea', 1), ('Holy See (Vatican City State)', 1), ('Algeria', 3), ('France', 4), ('Slovakia', 1), ('Israel', 4), ('Nauru', 1), ('Senegal', 1), ('Kenya', 2), ('Malaysia', 3), ('Zambia', 1), ('Hong Kong', 1), ('Kuwait', 1), ('Madagascar', 1), ('Philippines', 20), ('Tuvalu', 1), ('United States', 36), ('Turkey', 15), ('Nigeria', 13), ('China', 53), ('Belarus', 2), ('Armenia', 1), ('Netherlands', 5),

## 4. Display information about a store that has more than 100 customers and less than 300 customers.

In [11]:
mapper_stmt4 = select(
    dict_table['store'].c.store_id,
    dict_table['store'].c.manager_staff_id,
    dict_table['store'].c.address_id,
    func.count(dict_table['customer'].c.customer_id).label('customer_count')
).select_from(
    dict_table['store']
    .join(dict_table['customer'], dict_table['store'].c.store_id == dict_table['customer'].c.store_id)
).group_by(
    dict_table['store'].c.store_id
).having(
    func.count(dict_table['customer'].c.customer_id) > 100,
    func.count(dict_table['customer'].c.customer_id) < 300
)

print('Mapper select: ')
print(mapper_stmt4)

with engine.connect() as conn:
    result4 = conn.execute(mapper_stmt4).fetchall()

print(result4)

Mapper select: 
SELECT store.store_id, store.manager_staff_id, store.address_id, count(customer.customer_id) AS customer_count 
FROM store JOIN customer ON store.store_id = customer.store_id GROUP BY store.store_id 
HAVING count(customer.customer_id) > :count_1 AND count(customer.customer_id) < :count_2
[(2, 2, 2, 273)]


## 5. Select all customers who watched movies for more than 200 hours.

In [12]:
mapper_stmt5 = select(
    dict_table['customer'].c.customer_id,
    dict_table['customer'].c.first_name,
    dict_table['customer'].c.last_name
).select_from(
    dict_table['rental']
    .join(dict_table['inventory'], dict_table['rental'].c.inventory_id == dict_table['inventory'].c.inventory_id)
    .join(dict_table['film'], dict_table['inventory'].c.film_id == dict_table['film'].c.film_id)
    .join(dict_table['customer'], dict_table['rental'].c.customer_id == dict_table['customer'].c.customer_id)
).group_by(
    dict_table['customer'].c.customer_id
).having(
    func.sum(dict_table['film'].c.length) > 200 * 60  # 200 hours in minutes
)

print('Mapper select 5: ')
print(mapper_stmt5)

with engine.connect() as conn:
    result5 = conn.execute(mapper_stmt5).fetchall()

print(result5)

Mapper select 5: 
SELECT customer.customer_id, customer.first_name, customer.last_name 
FROM rental JOIN inventory ON rental.inventory_id = inventory.inventory_id JOIN film ON inventory.film_id = film.film_id JOIN customer ON rental.customer_id = customer.customer_id GROUP BY customer.customer_id 
HAVING sum(film.length) > :sum_1
[]


## 6. Calculate the average value of a movie rented.

In [13]:
mapper_stmt6 = select(
    func.avg(dict_table['film'].c.rental_rate).label('average_rental_value')
)

print('Mapper select 6: ')
print(mapper_stmt6)

with engine.connect() as conn:
    result6 = conn.execute(mapper_stmt6).fetchall()

print(result6)

Mapper select 6: 
SELECT avg(film.rental_rate) AS average_rental_value 
FROM film
[(Decimal('2.9800000000000000'),)]


## 7. Calculate the average value of the video length in all categories.

In [14]:
mapper_stmt7 = select(
    dict_table['category'].c.name,
    func.avg(dict_table['film'].c.length).label('average_length')
).select_from(
    dict_table['film_category']
    .join(dict_table['film'], dict_table['film_category'].c.film_id == dict_table['film'].c.film_id)
    .join(dict_table['category'], dict_table['film_category'].c.category_id == dict_table['category'].c.category_id)
).group_by(
    dict_table['category'].c.name
)

print('Mapper select 7: ')
print(mapper_stmt7)

with engine.connect() as conn:
    result7 = conn.execute(mapper_stmt7).fetchall()

print(result7)

Mapper select 7: 
SELECT category.name, avg(film.length) AS average_length 
FROM film_category JOIN film ON film_category.film_id = film.film_id JOIN category ON film_category.category_id = category.category_id GROUP BY category.name
[('Family', Decimal('114.7826086956521739')), ('Games', Decimal('127.8360655737704918')), ('Animation', Decimal('111.0151515151515152')), ('Classics', Decimal('111.6666666666666667')), ('Documentary', Decimal('108.7500000000000000')), ('New', Decimal('111.1269841269841270')), ('Sports', Decimal('128.2027027027027027')), ('Children', Decimal('109.8000000000000000')), ('Music', Decimal('113.6470588235294118')), ('Travel', Decimal('113.3157894736842105')), ('Foreign', Decimal('121.6986301369863014')), ('Drama', Decimal('120.8387096774193548')), ('Horror', Decimal('112.4821428571428571')), ('Action', Decimal('111.6093750000000000')), ('Sci-Fi', Decimal('108.1967213114754098')), ('Comedy', Decimal('115.8275862068965517'))]


## 8. Find longest movie titles in all categories.

In [15]:
mapper_stmt8 = select(
    dict_table['category'].c.name,
    dict_table['film'].c.title,
    dict_table['film'].c.length
).select_from(
    dict_table['film_category']
    .join(dict_table['film'], dict_table['film_category'].c.film_id == dict_table['film'].c.film_id)
    .join(dict_table['category'], dict_table['film_category'].c.category_id == dict_table['category'].c.category_id)
).where(
    dict_table['film'].c.length == select(func.max(dict_table['film'].c.length))
    .select_from(
        dict_table['film_category'].join(dict_table['film'], dict_table['film_category'].c.film_id == dict_table['film'].c.film_id)
    )
    .where(
        dict_table['film_category'].c.category_id == dict_table['category'].c.category_id
    ).scalar_subquery()
)

print('Mapper select 8: ')
print(mapper_stmt8)

with engine.connect() as conn:
    result8 = conn.execute(mapper_stmt8).fetchall()

print(result8)

Mapper select 8: 
SELECT category.name, film.title, film.length 
FROM film_category JOIN film ON film_category.film_id = film.film_id JOIN category ON film_category.category_id = category.category_id 
WHERE film.length = (SELECT max(film.length) AS max_1 
FROM film_category JOIN film ON film_category.film_id = film.film_id 
WHERE film_category.category_id = category.category_id)
[('Horror', 'Analyze Hoosiers', 181), ('Games', 'Chicago North', 185), ('Classics', 'Conspiracy Spirit', 184), ('Comedy', 'Control Anthem', 185), ('Foreign', 'Crystal Breaking', 184), ('Action', 'Darn Forrester', 185), ('New', 'Frontier Cabin', 183), ('Children', 'Fury Murder', 178), ('Animation', 'Gangs Pride', 185), ('Music', 'Home Pity', 185), ('Drama', 'Jacket Frisco', 181), ('Family', 'King Evolution', 184), ('Horror', 'Love Suicides', 181), ('Travel', 'Muscle Bright', 185), ('Animation', 'Pond Seattle', 185), ('Sports', 'Smoochy Control', 184), ('Sci-Fi', 'Soldiers Evolution', 185), ('Foreign', 'Sorority 

## 9. Find longest movie in all categories. Compare the result with point 10.

In [16]:
mapper_stmt9 = select(
    dict_table['category'].c.name,
    dict_table['film'].c.title,
    dict_table['film'].c.length
).select_from(
    dict_table['film_category']
    .join(dict_table['film'], dict_table['film_category'].c.film_id == dict_table['film'].c.film_id)
    .join(dict_table['category'], dict_table['film_category'].c.category_id == dict_table['category'].c.category_id)
).where(
    dict_table['film'].c.length.in_(
        select(func.max(dict_table['film'].c.length))
        .select_from(dict_table['film_category']
            .join(dict_table['film'], dict_table['film_category'].c.film_id == dict_table['film'].c.film_id)
        )
        .group_by(dict_table['film_category'].c.category_id)
    )
)

print('Mapper select 9: ')
print(mapper_stmt9)

with engine.connect() as conn:
    result9 = conn.execute(mapper_stmt9).fetchall()

print(result9)

Mapper select 9: 
SELECT category.name, film.title, film.length 
FROM film_category JOIN film ON film_category.film_id = film.film_id JOIN category ON film_category.category_id = category.category_id 
WHERE film.length IN (SELECT max(film.length) AS max_1 
FROM film_category JOIN film ON film_category.film_id = film.film_id GROUP BY film_category.category_id)
[('Horror', 'Analyze Hoosiers', 181), ('Foreign', 'Catch Amistad', 183), ('Games', 'Chicago North', 185), ('Classics', 'Conspiracy Spirit', 184), ('Comedy', 'Control Anthem', 185), ('Foreign', 'Crystal Breaking', 184), ('Action', 'Darn Forrester', 185), ('Sports', 'Drop Waterfront', 178), ('Sci-Fi', 'Express Lonely', 178), ('New', 'Frontier Cabin', 183), ('Children', 'Fury Murder', 178), ('Animation', 'Gangs Pride', 185), ('Games', 'Haunting Pianist', 181), ('Music', 'Home Pity', 185), ('Foreign', 'Hotel Happiness', 181), ('Sports', 'Image Princess', 178), ('Foreign', 'Innocent Usual', 178), ('Foreign', 'Intrigue Worst', 181), ('D